# 文本转换

在这个笔记本中，我们将探讨如何使用大型语言模型进行文本转换任务，如语言翻译、拼写和语法检查、语气调整和格式转换。

[website](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/6/transforming)

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

## 翻译
ChatGPT 接受许多种语言的训练资料。这使得模型具有翻译能力。以下是如何使用此功能的一些示例。

In [3]:
prompt = f"""
翻译以下英文文本为汉语: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

嗨，我想订购一个搅拌机。


In [4]:
prompt = f"""
告诉我这是哪种语言: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

这是法语。


In [6]:
prompt = f"""
翻译以下文本为法语/西班牙语和英语：
```我想买一个篮球```
"""
response = get_completion(prompt)
print(response)

法语：Je veux acheter un ballon de basket.
西班牙语：Quiero comprar una pelota de baloncesto.
英语：I want to buy a basketball.


In [8]:
prompt = f"""
将以下文本翻译成中文，正式和非正式形式：
“Would you like to order a pillow？”
"""
response = get_completion(prompt)
print(response)

正式形式：您是否需要订购一个枕头？

非正式形式：你想要订购一个枕头吗？


### 通用翻译器

想象一下，您是一家大型跨国电商公司的 IT 主管。用户用各种母语向您发送 IT 问题。您的员工来自世界各地，只说他们的母语。您需要一个通用翻译器！

In [10]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "My screen is flashing"                                      # My screen is flashing
] 

In [12]:
for issue in user_messages:
    prompt = f"告诉我下面文本的语言是: ```{issue}```"
    lang = get_completion(prompt)
    print(f"原始语言 ({lang}): {issue}")

    prompt = f"""
    将以下文本翻译成中文和英文：``{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

原始语言 (法语。): La performance du système est plus lente que d'habitude.
中文翻译：系统性能比平时慢。

英文翻译：The system performance is slower than usual. 

原始语言 (西班牙语。): Mi monitor tiene píxeles que no se iluminan.
中文翻译：我的显示器有一些像素点不亮。

英文翻译：My monitor has pixels that do not light up. 

原始语言 (意大利语。): Il mio mouse non funziona
中文翻译：我的鼠标不工作。

英文翻译：My mouse is not working. 

原始语言 (波兰语。): Mój klawisz Ctrl jest zepsuty
中文翻译：我的Ctrl键坏了。

英文翻译：My Ctrl key is broken. 

原始语言 (英语。): My screen is flashing
中文翻译：我的屏幕在闪烁。

英文翻译：My screen is flashing. 



## 语气转换

根据目标受众，写作可能有所不同。ChatGPT 可以生成不同的语气。


In [13]:
prompt = f"""
将以下俚语翻译成商业信函：
“老兄，我是乔，看看这个立灯的规格。”
"""
response = get_completion(prompt)
print(response)

尊敬的先生/女士，我是乔，想请您查看这个立灯的规格。


## 格式转换

ChatGPT 可以在格式之间进行转换。Prompt 应描述输入和输出格式。

In [14]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
将以下 Python 字典从 JSON 翻译成带有列标题和标题的 HTML 表格： {data_json}
"""
response = get_completion(prompt)
print(response)

<table>
  <tr>
    <th>resturant employees</th>
    <th>name</th>
    <th>email</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Shyam</td>
    <td>shyamjaiswal@gmail.com</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Bob</td>
    <td>bob32@gmail.com</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Jai</td>
    <td>jai87@gmail.com</td>
  </tr>
</table>


In [15]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

resturant employees,name,email
1,Shyam,shyamjaiswal@gmail.com
2,Bob,bob32@gmail.com
3,Jai,jai87@gmail.com


## 拼写检查/语法检查

以下是一些常见的语法和拼写问题以及 LLM 的回应。

要向 LLM 表明您希望它校对您的文本，您可以指示模型进行“校对”或“校对并纠正”。

In [18]:
text = [
"我昨天去图书馆借几本书籍。", # 借 -> 借了
"他们每天都去跑步锻炼身体。", # 锻炼身体 -> 锻炼
"这个苹果味道好吃极了。", # 好吃极了 -> 非常好吃
"我喜欢吃苹果和香蕉这些水果。", # 这些水果 -> （删除）
"这个手机的屏幕尺寸特大。", # 特大 -> 很大
"我昨天去超市买了一些东西。", # 去超市 -> 去了超市
"他的房子离公司路程很近。" # 路程很近 -> 很近
]

for t in text:
    prompt = f"""
    校对并更正以下文字并重写更正后的版本，并标注错误信息。 如果你没有找到错误，只需说“未发现错误”。 
    不要使用文本周围的任何标点符号：
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

未发现错误。

重写后的版本：昨天我去了图书馆借了几本书。
未发现错误。

重写后的版本：他们每天都去跑步来锻炼身体。
未发现错误。 

重写后的版本：这个苹果味道非常好。
未发现错误。

重写后的版本：我喜欢吃水果，尤其是苹果和香蕉。
未发现错误。

重写后的版本：这款手机的屏幕尺寸非常大。
未发现错误。

重写后的版本：昨天我去了超市买了一些东西。
未发现错误。

重写后的版本：他的房子离公司很近。


In [22]:
text = f"""
我买了这个给我女儿当她生日礼物，因为她总是从我房间拿走我的。是的，大人也喜欢熊猫。她带着它到处走，而且它超级柔软可爱。
其中一个耳朵比另一个低一点，我不认为这是设计成不对称的。虽然我觉得它对我付的钱来说有点小。我觉得可能有其他更大的选择，价格一样。
它比预期早到了一天，所以我在给女儿之前自己玩了一下。
"""
prompt = f"校对并更正此评论: ```{text}```"
response = get_completion(prompt)
print(response)

我买了这个作为我女儿的生日礼物，因为她总是从我的房间拿走我的熊猫。是的，成年人也喜欢熊猫。她带着它到处走，它非常柔软和可爱。其中一个耳朵比另一个低一点，我不认为这是故意设计成不对称的。虽然我觉得它对我付的钱来说有点小，我认为可能有其他更大的选择，价格相同。它比预期提前了一天送到，所以我在送给女儿之前自己玩了一下。


In [23]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style="color:red;font-weight:700;text-decoration:line-through;">我买了这个给我女儿当她生日礼物，因为她总是从我房间拿走我的。是的，大人也喜欢熊猫。她带着它到处走，而且它超级柔软可爱。
其中一个耳朵比另一个低一点，我不认为这是设计成不对称的。虽然我觉得它对我付的钱来说有点小。我觉得可能有其他更大的选择，价格一样。
它比预期早到了一天，所以我在给女儿之前自己玩了一下。
</span><span style="color:red;font-weight:700;">我买了这个作为我女儿的生日礼物，因为她总是从我的房间拿走我的熊猫。是的，成年人也喜欢熊猫。她带着它到处走，它非常柔软和可爱。其中一个耳朵比另一个低一点，我不认为这是故意设计成不对称的。虽然我觉得它对我付的钱来说有点小，我认为可能有其他更大的选择，价格相同。它比预期提前了一天送到，所以我在送给女儿之前自己玩了一下。</span>

In [25]:
prompt = f"""
校对并更正此评论。 使其更具吸引力。确保它遵循 金字塔写作原理 指南并面向高级读者。
以 markdown 格式输出。
文本内容: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

Title: 柔软可爱的熊猫玩具

我为女儿的生日礼物买了这个熊猫玩具，因为她总是从我的房间拿走我的东西。但是，我承认，我也很喜欢这个熊猫。它非常柔软和可爱，女儿带着它到处走。

虽然其中一个耳朵比另一个低一点，但我不认为这是设计上的不对称。但是，考虑到我付的钱，我觉得它有点小了。也许有其他更大的选择，价格也相同。

不过，这个熊猫比我预期的早到了一天，所以我在女儿拿到它之前就先玩了一下。总的来说，这是一个非常可爱的玩具，我相信女儿会喜欢它。

感谢以下网站：

https://writingprompts.com/bad-grammar-examples/
